# SecSSE Body-size

In [ ]:
# SecSSE Body-size

## I Data preparation

In [ ]:
# I Data preparation

### Loading packages

In [ ]:
# Loading packages
library("ape")
library("secsse")
library("DDD")
library("doMC")
library("tidyverse")
library("parallel")
library("qgraph")
library("stringr")

### Loading data

In [ ]:
# Loading data
df<-read.csv("Table_body_size.tsv", sep="\t") # omit sep ="\t" for .csv files
phy1.0<-read.nexus("382sp_16C_20FC.tree")

### Optional step (cleaning data)

In [ ]:
# Optional step (cleaning data)
states<-df$Maximum_body_size
names(states)=str_replace((rownames(df)), " ", "_")
states2<-states[!names(states) %in% setdiff(names(states), phy1.0$tip.label)]

### Computing sampling fractions

In [ ]:
# Computing sampling fractions
f<-c(
length(states2[states2=="1"])/length(df$Maximum_body_size[df$Maximum_body_size=="1"]),
length(states2[states2=="2"])/length(df$Maximum_body_size[df$Maximum_body_size=="2"]),
length(states2[states2=="3"])/length(df$Maximum_body_size[df$Maximum_body_size=="3"]))

In [ ]:
states3<-as.data.frame(cbind(names(states2), as.factor(states2)))
colnames(states3)<-c("species","states")
rownames(states3)<-NULL
traits <- sortingtraits(states3, phy)

In [ ]:
n_states<-length(unique(na.omit(traits)))

In [ ]:
states<-traits

## II SecSSE analyses

In [ ]:
# II SecSSE analyses

### Loads the models to be tested

In [ ]:
# Loads the models to be tested
source("aux_scripts/secsse_functions.R")
source("aux_scripts/secsse_base_models.R")

### Constraining some transitions to be null

In [ ]:
# Constraining some transitions to be null
mask<-matrix(1, nrow=3, ncol=3)
diag(mask)<-NA
mask[1,3]<-0
mask[3,1]<-0

### Applying the previous matrix

In [ ]:
# Applying the previous matrix
for(i in 1:length(models)){
  models[[i]]$idparslist$Q<-mask_q(q=models[[i]]$idparslist$Q, mask=mask, n_states = n_states)
}

for(i in 1:length(models)){
  
  idparsopt = sort(unique(na.omit(unname(unlist(models[[i]]$idparslist)))))
  if(any(idparsopt %in% 0)){
    idparsopt = idparsopt[idparsopt!=0]
    idparsfix = c(0)
    parsfix = c(0)
  } else {
    idparsfix = c()
    parsfix = c()
  }
  
  models[[i]]<-append(models[[i]],
                      list(
                        idparsopt=idparsopt,
                        idparsfix=idparsfix,
                        parsfix=parsfix
                      )
  )
}

### Assigning starting value and initialization

In [ ]:
# Assigning starting value and initialization
startingpoint <- bd_ML(brts = ape::branching.times(phy))
intGuessLambda <- startingpoint$lambda0
intGuessMu <- startingpoint$mu0

intGuessLambdas<-c(intGuessLambda, intGuessLambda/2, intGuessLambda*2)
intGuessMus <- c(intGuessMu, intGuessMu/2, intGuessMu*2)
initTrans <- intGuessLambdas/n_states

nrep=length(intGuessLambdas)

replicate_models<-rep(models,each=nrep)
names(replicate_models)<-paste0(rep(names(models), each=nrep), rep(paste0("_try", 1:nrep),nrep))
names(replicate_models)
models<-replicate_models
iterator=rep(1:nrep, length(models)/nrep)
for(i in 1:length(models)){
  initparsopt = c(
    rep(intGuessLambdas[iterator[i]],length(unique(models[[i]]$idparslist$lambdas))),
    ifelse(unique(models[[i]]$idparslist$mus)!=0,
      rep(intGuessMus[iterator[i]],length(unique(models[[i]]$idparslist$mus))),
      0
    ),
    rep(initTrans[iterator[i]],length(which(unique(c(models[[i]]$idparslist$Q))>0)))
  )
  initparsopt<-initparsopt[initparsopt!=0]

  models[[i]]$initparsopt<-initparsopt

}

all(sapply(models, FUN = function(x) length(x$initparsopt)==length(x$idparsopt)))

lapply(models, `[[`, "idparslist")

In [ ]:
models<-lapply(models, FUN=append,
               list(phy = phy,
                    traits = states,
                    cond="proper_cond",
                    root_state_weight = c(1,0,0),
                    sampling_fraction=f,
                    num_cycles = 3, ### WARNING!!!! set this to something higher!!! - takes a very long time
                    run_parallel=T)
               )

### Creating an output directory

In [ ]:
# Creating an output directory
run_secsse<-function(i) {

  if(!dir.exists("secsse_out_body_size")) dir.create("secsse_out_body_size")

  try(
    saveRDS(do.call(secsse_ml, args=models[[i]]),
            file = paste0("secsse_out_body_size/",names(models)[i],"_out.rds")))
}

### Running the models in parallel

In [ ]:
# Running the models in parallel
n_cores=ifelse(length(models)>30, 30, length(models))
n_cores
mclapply(FUN=run_secsse,
         X=1:length(models),
         mc.cores = n_cores)